In [1]:
import torch
from sparse_ae import *
#import matplotlib.pyplot as plt
from scipy.misc import imresize
from torch.autograd import Variable
import sys
import numpy as np
%matplotlib inline
sys.path.append('../Localization/')
from torchvision import datasets,models,transforms
import os
#from image import readImage
from utils import *

In [2]:
# temp_im = plt.imread('/data/gabriel/LVseg/dataset_img/img_256'+'/'+'DET0040201_SA9_ph2.png')
# dim = temp_im.shape
# temp_im2 = temp_im.reshape(1,dim[0],dim[1])
# import torchvision
# a = torch.Tensor(temp_im2)
# #a = torchvision.transforms.ToTensor()(temp_im2)

In [ ]:
img_path2 = '/data/gabriel/LVseg/patches/Training'
total_im = sum([1 for i in os.listdir(img_path2) if '.png' in i])
epoch_sae = [2000,5000]
# for e_sae in epoch_sae:
#     a = SAE(n_in=11,n_h=100,n_out=11,img_path=img_path2,b_size = total_im,patch_size=11,lr = 0.001,gpu=2)
#     w,b = train_sae(a,epochs = e_sae)
#     with open('/data/gabriel/LVseg/'+str(e_sae)+'.p','wb') as f:
#         pickle.dump([w,b],f)
    


In [ ]:
#img_path = '/data/gabriel/LVseg/dataset_img/img/Training'

pre_train_sae = [True,False]
pre_train_cl = [True,False]
epoch_cl = [5000,10000,10**5]
#epoch_cl = [3,1,2]

cl_lr = [0.1,0.01,0.001,0.0001,0.00001]

for cl_lrs in cl_lr:
    for e_cl in epoch_cl:
        for p_sae in pre_train_sae:
            for p_cl in pre_train_cl:
                nn_l = localnet(img_path='/data/gabriel/LVseg/dataset_img/img_256/',label_path='/data/gabriel/LVseg/dataset_img/box_256/',b_size=999,test_fraction=0.1,lr = cl_lrs,gpu=1)

                if(p_sae):
                    for e_sae in epoch_sae:
                        with open('/data/gabriel/LVseg/'+str(e_sae)+'.p','rb') as ff:
                            [w,b] = pickle.load(ff)
                        for p in nn_l.modules():
                            if isinstance(p,nn.Conv2d):
                                #print(p.weight.data)
                                #print(p.bias.data)
                                p.weight.data, p.bias.data= w.view(100,1,11,11),b
                                break
                        save_dir_name = '/data/gabriel/LVseg/'+str(p_sae)+'_'+str(p_cl)+'_'+str(e_sae)+'_'+str(e_cl)
                        model_name = str(p_sae)+'_'+str(p_cl)+'_'+str(e_sae)+'_'+str(e_cl)+'.pth.tar'
                        os.makedirs(save_dir_name)
                        train_lnet(model=nn_l,epochs=e_cl,fname = save_dir_name+'/'+model_name,pretrain=p_cl,save_dir=save_dir_name)
                        
                        test_lnet(model=nn_l,fname=save_dir_name+'/'+model_name)
                else:
                    save_dir_name = '/data/gabriel/LVseg/'+str(p_sae)+'_'+str(p_cl)+'_'+str(e_sae)+'_'+str(e_cl)
                    model_name = str(p_sae)+'_'+str(p_cl)+'_'+str(e_sae)+'_'+str(e_cl)+'.pth.tar'
                    
                    os.makedirs(save_dir_name[:save_dir_name.find('.png')])
                    train_lnet(model=nn_l,epochs=e_cl,fname = save_dir_name+'/'+model_name,pretrain=p_cl,save_dir=save_dir_name)

                    test_lnet(model=nn_l,fname=save_dir_name+'/'+model_name)


In [ ]:
abc = 'abv3'
print(abc[:abc.find('b')+3])

In [ ]:
#out = output.data

In [ ]:
#out1 = out.numpy()

In [ ]:
#out1 = out1.reshape(121,121)

In [ ]:
#plt.imshow(out1),plt.show()

In [ ]:
#out = output.numpy().transpose(1,2,0)